# Airbnb NYC 2019


In [ ]:
import pandas as pd
import numpy as np

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
mpl.rcParams['font.size'] = 12
mpl.rcParams['axes.labelsize'] = 10

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Data Exploration
Load data and perform first exploration

In [ ]:
df_airbnb = pd.read_csv('./Data/AB_NYC_2019.csv')

In [ ]:
df_airbnb.head()

In [ ]:
print('Number of entries: {} \nNumber of features: {}'.format(df_airbnb.shape[0], df_airbnb.shape[1]))

A first look suggests that columns such as **id**, **name**, **host_id** and **host_name** can be discarded from the analysis. 

The column **name** could be used to incorporate some more features (keywords appearing in the name). This will be discarded on the first simple models.

In [ ]:
# drop features id, host_id and host_name
df_airbnb.drop(['id', 'name', 'host_name'], axis=1, inplace=True)

Now we will look for missing data on the remaining features

In [ ]:
df_airbnb.isnull().sum()

We have missing data for the following features:
* last_review --> comes from entries without any review
* reviews_per_month --> comes from entries without any review; we will replace it by 0

In [ ]:
# replace NaN by 0 for column reviews per month
df_airbnb.reviews_per_month.fillna(0, inplace=True)

So far we have 12 columns, 11 of them corresponding to features and 1 corresponding to our target variable, price. 

First we will start by taking a look to the price to see its distribution and check there are no inconsistencies.

Then we will explore the features and their distributions and correlation among them. We can expect high correlation between:
* neighbourhood_group, neighbourhood, latitude and longitude
* number_of_reviews and reviews_per_month (can also be related with last_review)

### Exploration of price

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(15,5))
df_airbnb['price'].plot(kind='hist', bins=80, title='Price distribution', ax=ax[0])
df_airbnb['price'].plot(kind='hist', bins=80,logx=True, logy=True, title='Price distribution', ax=ax[1])
ax[0].set_xlabel('Price $')
ax[1].set_xlabel('Log Price $')
plt.show()

In [ ]:
df_airbnb['price'].describe()

In [ ]:
print('Number of cases with price = $0: ',len(df_airbnb[df_airbnb.price == 0]))

In [ ]:
# drop cases with price 0
df_airbnb.drop(df_airbnb[df_airbnb.price < 10]. index, axis=0, inplace=True)

In [ ]:
def print_pct_price(attribute, value):
    print('Appartments with {} over ${}: {:.2f}%'.\
          format(attribute, \
                 value, \
                 100 * len(df_airbnb[df_airbnb[attribute] > value])/len(df_airbnb)))

print_pct_price('price', 200)
print_pct_price('price', 300)
print_pct_price('price', 500)
print_pct_price('price', 1000)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(15,5))
df_airbnb[df_airbnb['price'] <= 400].price.plot(kind='hist', bins=100,title='Price distribution below $300', ax=ax[0])
df_airbnb[df_airbnb['price'] > 400].price.plot(kind='hist', bins=100, title='Price distribution above $300', ax=ax[1])
ax[0].set_xlabel('Price $')
ax[1].set_xlabel('Price $')
plt.show()


Insights: 
* Mean price is \\$153 and median price is \\$106. This reflects large outliers that increase the mean price.
* Min price is \\$0 while max price is \\$10k. We will drop the cases with price = \\$0 since they may correspond to errors.
* Around 95% of the appartments have a price lower than \\$300. From these, a good amount concentrate in a range lower than \\$100. We can also observe some peaks around round prices (100, 150, etc.)
* Appartments over \\$1k represent less than 0.5\% of the cases.

### Exploration of features

The features **neighbourhood** is left out of the plot due to the large number of possible values. The feature **last_review** is left out of the plot since we cannot relate it to the date when the prices were retrieved.

In [ ]:
f, ax = plt.subplots(3,3, figsize=(40,30))
ax = ax.flatten()
for idx, col in enumerate(df_airbnb.drop(['neighbourhood', 'price', 'last_review'], axis=1).columns):
    if df_airbnb[col].dtype == object:
        df_airbnb.groupby(col).count().iloc[:,1].plot(kind='bar', ax=ax[idx], title=col)
        ax[idx].tick_params(axis='x', rotation=0)
    else:
        df_airbnb[col].plot(kind='hist', bins=50, ax=ax[idx], title=col)

Insights: 
* Neighbourhood: Brooklyn and Manhattan are the most represented ones.
* Most locations offered are entire appartments or private rooms. Shared rooms are very rare.
* Number of reviews and reviews per month seem to follow a Poisson distribution.
* Minimum nights and host listing count may offer more insights when taking logarithm of the values. 
* Availability is highly concentrated aroud 1 and then there is a great decrease for the rest of values.

#### Analysis per neighbourhood

In [ ]:
df_airbnb.groupby('neighbourhood_group').agg({'neighbourhood':['count','nunique'], 
                                              'price':['min', 'mean', 'median', 'max'],
                                             'number_of_reviews':['min', 'mean', 'median', 'max']})

If we analyse by neighbourhood, we can highlight:
* Manhattan is the one grouping less neighbourhoods with around 60% of the neighbourhoods that Queens groups (which is the one with the largest value).
* In terms of price, they all have almost the same minimum price. Nevertheless, Manhattan shows a median price that is almost 3 times the median of the lowest price (Bronx). The maximum price at Bronx is 25% of the maximum at Manhattan, Brooklyn and Staten Island. 
* In terms of number of reviews, the median is much lower than the mean in all cases, showing the presence of large sporadic numbers and a great amount of low values.

In [ ]:
def plot_price_dist(col_name):
    f, ax = plt.subplots(1,2, figsize=(20,5))
    ax = ax.flatten()

    sns.violinplot(data=df_airbnb[df_airbnb.price < 350], x=col_name, y='price', ax=ax[0])
    sns.violinplot(data=df_airbnb, x=col_name, y='price', ax=ax[1])
    ax[0].set_title('Price (up to $350) vs {}'.format(col_name))
    ax[1].set_title('Price vs {}'.format(col_name))
    plt.show()

In [ ]:
plot_price_dist('neighbourhood_group')

#### Analysis type of room

In [ ]:
df_airbnb.groupby('room_type').agg({'price':['min', 'mean', 'median', 'max'],
                                    'number_of_reviews':['min', 'mean', 'median', 'max']})

In [ ]:
plot_price_dist('room_type')

From the type of room, we can highlight:
* Private and Share rooms have a similar mean price but private rooms can increase to much higher prices ( \\$10k vs \\$1.8k) 

#### Analysis reviews
The columns **last review** does not offer us much data since this value is static and we cannot relate it to the current date (the data has not been updated and we don't know the day when it was generated). Therefore, we will not consider it. 

We expect high correlation between **number of reviews** and **reviews per month**

In [ ]:
df_airbnb.drop(['last_review'], axis=1, inplace=True)
df_airbnb.head()

In [ ]:
f, ax = plt.subplots(figsize=(20,5))
df_airbnb.plot(kind='scatter', x='number_of_reviews', y='reviews_per_month', ax=ax)
plt.show()

In [ ]:
f, ax = plt.subplots(1,2,figsize=(20,5))
df_airbnb.plot(kind='scatter', x='number_of_reviews', y='price', \
               ax=ax[0], title='Price vs Number of Reviews')
df_airbnb.plot(kind='scatter', x='reviews_per_month', y='price', \
               ax=ax[1], title='Price vs Reviews/Month')

plt.show()

Insights from reviews:
* Number of reviews and reviews per month are correlated although less than expected. The relation is bounded. 
* The relation with price shows an exponential decay.

#### Correlation between features and encoding of categorical variables

In [ ]:
# drop neighbourhood prioritizing neighbourhood group
df_airbnb.drop(['neighbourhood'], axis=1, inplace=True)
# transform categorical variables to 1-hot encoding
df_airbnb_encoded = pd.get_dummies(df_airbnb)

In [ ]:
#corr = np.corrcoef(df_airbnb_encoded.transpose()).round(decimals=3)
#fig, ax = plt.subplots(figsize=(20,20))
#sns.heatmap(corr, vmin=-1, vmax=1, annot=True, ax=ax, \
#            xticklabels=df_airbnb_encoded.columns, yticklabels=df_airbnb_encoded.columns)
#plt.show()

In [ ]:
df_airbnb_encoded

## TODO

## Price Prediction:

* Create train and test set
* Normalize features
* For different models:
    - Train, tune hyperparameters, evaluate performance


In [ ]:
df_outliers = df_airbnb_encoded.drop(df_airbnb_encoded[df_airbnb_encoded['price'] > 350].index, axis=0)
#df_outliers = df_airbnb_encoded
X, y = df_outliers.drop(['price'], axis=1), df_outliers.price

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# scale features
# Not using StandardScaler to treat differently continuous, discrete and categorical variables

def scale(x, df_train, df_test, cont=True):
    std = np.std(df_train[x])

    if cont:
        mean = np.mean(df_train[x])
        df_train[x] = df_train[x].apply(lambda x: (x - mean)/std)
        df_test[x] = df_test[x].apply(lambda x: (x - mean)/std)
    else:
        median = np.median(df_train[x])
        df_train[x] = df_train[x].apply(lambda x: (x - median)/std)
        df_test[x] = df_test[x].apply(lambda x: (x - median)/std)


X_train_transformed = X_train.copy()
X_test_transformed = X_test.copy()

for feature in ['latitude', 'longitude', 'reviews_per_month']:
    scale(feature, X_train_transformed, X_test_transformed)


for feature in ['minimum_nights', 'number_of_reviews', 'calculated_host_listings_count', 'availability_365']:
    scale(feature, X_train_transformed, X_test_transformed, cont=False)  



## Define metrics

In [ ]:
def get_score(y_pred, y_true):
    print('RMSE: {:.4f}'.format(mean_squared_error(y_true, y_pred, squared=False)))
    print('MAE : {:.4f}'.format(mean_absolute_error(y_true, y_pred)))
    print('R2  : {:.4f}'.format(r2_score(y_true, y_pred)))


In [ ]:
mask_outliers = y_train <= 350

#### Model 1. Ridge Regression --REVIEW!! 

In [ ]:
params1 = np.logspace(-3, 3, 40)

model1 = Ridge()
grid1 = GridSearchCV(estimator=model1, param_grid=dict(alpha=params1), cv=5)
grid1.fit(X_train_transformed, y_train)

y_pred1 = grid1.predict(X_test_transformed)
get_score(y_pred1, y_test)
print(grid1.best_params_)

#### Model 2. Elastic Net

In [ ]:
params2 = {'alpha':np.logspace(-2, 2, 10), 'l1_ratio':np.linspace(1e-2, 1, 10)}

model2 = ElasticNet(max_iter=100)
grid2 = GridSearchCV(estimator=model2, param_grid=params2, cv=5)
grid2.fit(X_train_transformed, y_train)

y_pred2 = grid2.predict(X_test_transformed)
get_score(y_pred2, y_test)
print(grid2.best_params_)

#### Model 3. Random Forest Regressor

In [ ]:
params3 = {'max_depth': range(3,7), 'n_estimators': (10, 50, 100, 1000)}

model3 = RandomForestRegressor()
grid3 = GridSearchCV(estimator=model3, param_grid=params3, cv=5, scoring='neg_mean_squared_error')

grid_result = grid3.fit(X, y)
best_params = grid_result.best_params_

model3_best = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],
                                random_state=False, verbose=False)
model3_best.fit(X_train_transformed, y_train)
y_pred3 = model3_best.predict(X_test_transformed)
get_score(y_pred3, y_test)

In [ ]:
print('best_parameters: ', best_params)

#### Model 4. MLP

In [ ]:
params4 = [{'hidden_layer_sizes': [10,30,50, 100],
'activation': ['relu'],
'solver':['lbfgs'], 'alpha':[5e-1, 1e-2, 1e-3, 1e-4],
'batch_size':['auto'], 
'learning_rate_init':[1e-3], 'max_iter':[500]}]

model4 = MLPRegressor(verbose=True)
grid4 = GridSearchCV(model4, params4, cv=5)
grid_result = grid4.fit(X_train_transformed, y_train)
 
#mlp = MLPRegressor(hidden_layer_sizes=(20,), activation='logistic', solver='lbfgs', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, max_iter=500)
#mlp.fit(X_train,y_train)
train_mse = mean_squared_error(y_train, grid4.predict(X_train_transformed))
test_mse = mean_squared_error(y_test, grid4.predict(X_test_transformed))

y_pred4 = grid4.predict(X_test_transformed)
get_score(y_pred4, y_test)
 
print(grid4.best_params_)
print(grid4.best_score_)
print("Train MSE:", np.round(train_mse,2))
print("Test MSE:", np.round(test_mse,2))

In [ ]:
plt.figure(figsize=(20,5))
def error(y, y_pred):
    return np.sqrt((y - y_pred)**2)

plt.hist(error(y_test, y_pred1), bins=100, color='blue', alpha=0.5)
plt.hist(error(y_test, y_pred2), bins=100, color='red', alpha=0.5)
plt.hist(error(y_test, y_pred3), bins=100, color='green', alpha=0.5)
plt.hist(error(y_test, y_pred4), bins=100, color='black', alpha=0.2)
plt.legend(['Ridge', 'ElasticNet', 'RandomForest', 'MLP'])
plt.show()